RNN으로 영화평 감성 분석하기 2022년 1월 10일 김이룸

In [ ]:
# 1번 셀

# 패키지 수입
import numpy as np
from keras.datasets import imdb # 이미 데이터 처리가 되어있음
from keras.preprocessing.sequence import pad_sequences
from time import time

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.metrics import confusion_matrix, f1_score # f1 점수는 혼동행렬과 같이 따라다닌다.

In [ ]:
# 2번 셀

# 하이퍼 파라미터 설정
# 사전 안에 단어 수 (사용 빈도수로 sorting)
# 가장 많이 사용되는 단어 : the
# 단어수(MY_WORDS) 많다고 무조건 좋은가? 답 : no. 파라매터가 34만개로 늘어났다. 많이 안쓰는 단어도 학습해서 별로 효과를 보지 못했다.
MY_WORDS = 5000       # 5000 -> 10000사전 안에 단어 수 : 임의로 선택한 단어가 아니다. 특정한 조건을 갖고 선택한 단어들이다. 가장 많이 사용된 단어들이다. (기억하시오) -> 설계도 상 10000으로 표가된 수.
# 영화평 통일된 길이 -> 길게 해봐라
MY_LEN = 80           # 80 -> 200영화평 길이 : 길이란 단어 수. 각각의 영화평마다 길이가 다르다. 입력의 크기가 다르면 학습을 못한다. 입력을 통일해야한다. 모든 단어를 80개에서 잘러? -> 튜닝해서 재미봤다.
MY_EMBED = 32          # 임배딩 차원, 
MY_HIDDEN = 64         # LSTM의 차원 : 숫자 계산해서 오른쪽으로 몇개 넘겨주는지(여행자수 300개) 

MY_EPOCH = 10         # 반복 학습 수: 10 -> 100 : 튜닝해서 별로 재미 못봄
MY_BATCH = 200          # 한번에 처리하는 데이터 수 -> 학습속도와 관련이 있다./

요점정리


1.  Token 처리
2.   항목 추가



In [ ]:
# 3번 셀

# 데이터 불러오기 : keras가 사분할 해놨다.
# num_words : 사전에 단어를 몇개 쓸거냐
(X_train, Y_train),(X_test, Y_test) = imdb.load_data(num_words=MY_WORDS) # 총 5만개 들어있음. 학습용 2.5만개, 평가용 2.5만개.

# 샘플 출력 : 엑셀파일이 아니니 여기서 찎어야함
# IMDB의 영화평은 token 처리 되어 있음
# token : 1, 14, 12 각각의 숫자가 token. -> 영어단어 하나하나가 고유의 정수(토큰번호로) 대응되었다.
print('학습용 데이터 0번 영화평 : ', X_train[0]) # 영화평이라면서 숫자만 있네? : 답 : 영화평 맞다.
# 각각의 숫자는 무엇을 의미하는 걸까? 답 : 단어를 숫자로 치환한 것이다. 어떤 단어인지는 모른다. 어떤 단어인지 알아내기 위해서는 다음과 같은 과정을 거쳐야 한다.
print('학습용 데이터 0번 감성 : ',Y_train[0]) # 감성 : 0은 부정, 1은 긍정

# 기계는 단어로 학습 못한다. 숫자로는 학습할 수 있다. -> 영화평을 token 처리하면 기계가 학습할 수 있다.

# 데이터 모양 확인
print('학습용 입력 데이터 모양 : ',X_train.shape) # (25000,)은 1차원 데이터다. 0번 영화평 같은게 2만 5천개 있다.
# 영화평 데이터는 2차원 데이터가 아니다. 1차원 데이터이다. 중요한 컨셉이다. 판서를 참고하시오. -> 명확하게 이해가 안된다!? keras가 4분할 했는데 1차원인 데이터가 정확히 어느것인가? X_train이 영화평 데이터 인가? 맞다. X_train에 영화평, Y_train에 해당 영화평에 대한 감성분석 라벨 데이터가 있다.
# 하나의 데이터가 영화평 하나이다.
# 1차원인 이유 : 영화평 길이가 다름.
print('학습용 출력 데이터 모양 : ',Y_train.shape)
# print(Y_train) # 영화평이 이진수로 표현되어 있다.

print('평가용 입력 데이터 모양 : ',X_test.shape)
print('평가용 출력 데이터 모양 : ',Y_test.shape) # imdb는 학습용 반, 평가용 반으로 나눠져 있다.

17473536/17464789 [==============================] - 0s 0us/step
학습용 데이터 0번 영화평 :  [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 1

In [ ]:
# 4번 셀

# 단어를 정수로 전환하기 
word_to_id = imdb.get_word_index() # token 처리를 위해 사용한 파이썬 사전
print('the의 token : ', word_to_id['the']) # the를 넣으면 the의 token이 나온다.
print('virus의 token : ', word_to_id['virus']) # 코비드 전에는 3310보다 큰 숫자였다.


# 정수를 단어로 전환하기 : : token 처리된 영화평을 인간이 이해할 수 있게 전환하기 -> imdb에서 제공하지 않는다. 직접 구현해야 하나 어렵지 않다/
# word_to_id에는 단어들의 정수가 사용빈도수 대로 다 있다
# 사전을 처음부터 끝까지 
id_to_word = {} #  python 딕셔너리
for  word, id in word_to_id.items():
  id_to_word[id] = word

print('token 1의 단어 : ', id_to_word[1])
print('token 2의 단어 : ', id_to_word[2])
print('token 3310의 단어 : ', id_to_word[3310])

1654784/1641221 [==============================] - 0s 0us/step
the의 token :  1
virus의 token :  3310
token 1의 단어 :  the
token 2의 단어 :  and
token 3310의 단어 :  virus


In [ ]:
# 5번 셀

# 영화평 단어로 전환
# 영화평에 특수 숫자(0,1,2,) 3개 추가되어 있음 -> the가 1이 아니라 the가 4야. 특수숫자 3개(0,1,2)가 사전 앞에  더해져서. 3을 다 빼야 돼. -> 그건 어떻게 하느냐? 
# 0 : 패딩 -> 자리가 비면, 비어있으면  0으로 채워버리는 것. 
# 1 : 영화평 시작 -> 1은 영화평 시작이라는 의미다. 이런 의미의 특수숫자다.
# 2 : 삭제 단어 -> 영화평 0번에서 2를 찾을 수 있다. 2는 삭제된 단어를 의미한다.
def decode(review):
  output = []

  for i in review:
    
    # word = id_to_word[i-3] # 대괄호다. 소괄호 아니다. -> 3을 빼는 거다. 1에서 3을 빼서 에러가 생긴다. -> 0,1,2에서만 3빼면 안된다.
    word = id_to_word.get(i-3, '???') # 제대로 나오려면 3을 뺴야하는데. 음수가 나오면 문제가 발생한다. 음수가 발생해서 생긴 문제는 get함수로 처리하면 된다.
    output.append(word)
  print(output)
# dictionary.get() : dict의 Key로 Value얻기 # https://wikidocs.net/16
# decode(X_train[0]) # brilliant -> pos # 말이 안된다. 이게 영화평이라며. 전환했는데 왜 말이 안되는가? 답 : 영화평에 특수 문자 3개 추가되어 있어서.-> 0,1,2
decode(X_train[333]) # junk acting, bad director -> neg

['???', 'with', 'title', 'like', 'this', 'you', 'know', 'you', 'get', 'pretty', 'much', 'lot', 'of', 'junk', 'acting', 'bad', 'script', 'bad', 'director', 'bad', '???', 'bad', 'br', 'br', 'movie', 'make', 'lot', 'of', 'noise', 'that', 'really', 'not', 'music', 'and', 'lot', 'of', 'people', '???', 'movie', 'make', 'bad', 'racial', 'stereotype', 'why', 'come', 'every', 'movie', 'with', 'black', 'hero', 'have', 'drug', '???', 'why', 'come', 'hero', 'always', 'have', 'to', 'dance', 'to', 'be', 'success', 'why', 'come', 'famous', '???', 'always', 'have', 'to', 'be', 'in', 'dance', 'movie', 'why', 'come', 'letter', 's', "can't", 'be', 'in', 'title', 'br', 'br', 'hollywood', 'need', 'to', 'stop', 'dumb', 'down', 'audience', 'and', 'make', 'movie', 'that', 'have', 'people', 'with', 'brain', 'who', 'know', 'how', 'speak', 'proper', 'english', 'br', 'br', 'do', 'self', 'favor', 'and', 'not', 'go', 'see']


In [ ]:
# 6번 셀


# 영화평 길이가 천차만별 -> 길이가 다르면 RNN에 넣어줄 수가 없다. 길이를 통일시켜줘야 입력할 수 있다.
for i in range(10):
  print('영화평 : ', i, '길이 :', len(X_train[i]))

# 영화평 길이 통일
# truncate v. 자르다
# truncating='pre' : 앞부분 자르기
# truncating='post' : 뒷부분 자르기
# padding='pre' : 앞부분에 패딩처리. 앞부분을 (CNN padding='same'처럼)0으로 채우는 것.
# padding='post' : 뒷부분에 패딩처리. 뒷부분을 0으로 채우는 것.
# maxlen : 통일할 영화평의 길이 지정
X_train = pad_sequences(X_train,
                        truncating='pre',
                        padding='pre',
                        maxlen=MY_LEN) # 긴거는 자르고, 짦은 거는 채우기
print()
print()
for i in range(10):
  print('영화평 : ', i, '길이 :', len(X_train[i])) # 영화평 길이가 통일되었음을 확인할 수 있다.

X_test = pad_sequences(X_test,
                        truncating='pre',
                        padding='pre',
                        maxlen=MY_LEN) # 긴거는 자르고, 짦은 거는 채우기

영화평 :  0 길이 : 218
영화평 :  1 길이 : 189
영화평 :  2 길이 : 141
영화평 :  3 길이 : 550
영화평 :  4 길이 : 147
영화평 :  5 길이 : 43
영화평 :  6 길이 : 123
영화평 :  7 길이 : 562
영화평 :  8 길이 : 233
영화평 :  9 길이 : 130


영화평 :  0 길이 : 80
영화평 :  1 길이 : 80
영화평 :  2 길이 : 80
영화평 :  3 길이 : 80
영화평 :  4 길이 : 80
영화평 :  5 길이 : 80
영화평 :  6 길이 : 80
영화평 :  7 길이 : 80
영화평 :  8 길이 : 80
영화평 :  9 길이 : 80


In [ ]:
# 7번 셀 : # 패턴이 이제 보여야 한다. -> 순서는 다 똑같다.

# 데이터 모양 확인 : 인공신경망에 넣기 위함. 
print('학습용 입력 데이터 : ', X_train.shape) # 2차원 되었다. 왜? 길이 맞췄으니까.
print('학습용 출력 데이터 : ', Y_train.shape) # 그대로 1차원
print('평가용 입력 데이터 : ', X_test.shape) # 2차원 되었다. 왜? 길이 맞췄으니까.
print('평가용 출력 데이터 : ', Y_test.shape) # 그대로 1차원

# 인공신경망 구현하기 전에 설계도를 참고하라. : 여행자RNN 모델과 영화평RNN모델을 비교하시오. 공통점, 차이점 각각은 무엇인가.
# -> 영화평모델도 many-to-one 모델이다
# 맨 오른쪽에서 출력 - 공통
# Dense와 sigmoid- 공통
# 영화평 sigmoid: 이중분류. 0과 1일 각각의 확률 -> 0에 가까우면 0이고, 1에 가까우면 1이다
# 여행자 수 모델과의 차이점 : embed가 있다, 없다.
# 영화평 모델 RNN 설계도는 80단이다.(단어80) 여행자 RNN모델은 4단이다.
# 영화평모델 - 5000에서 32로 뚝 떨어진다. 
# 영화평 하나주고 감성 맞추고, 영화평 하나주고 감성 맞추고를 반복한다.

# 영화평 설계도 중 주목할 숫자 : 10000, 2
# 단어 한개가 1만개, 우리 코드에서는 5000이 되었다. 갑자기 왜 불어났을까? 답: 
# 토큰처리된 정수 1개가 어떻게 10000개로 늘어났는가? 답 : 원핫인코딩 -> 단어임베딩 강의자료를 참고하시오. (굉장히 중요) -> 2차원 임베딩
# 숫자가 1만개 들어간다. 왜 1만개가 들어가는가? 답: 
# 단어임베딩 : 임베딩은 소수다. 토큰 한자리 은정수다. (key 열 숫자는 token을 의미)
# 토큰화하면 숫자 하난데 왜 숫자를 더 많이 써서 단어 표현을 하느냐? 
# 원핫인코딩 : 정수를 이진수로 바꾸는 것. 자릿수는 정하기 나름. -> 단어임베딩의 유용성 강의자료
# 자연어 처리에서 왜 원핫인코딩을 하는가?
# 기계는 정수도 모른다. 이진수만 안다. 왜? 디지털 컴퓨터니까.
# 여러분이 안해도 케라스가 원핫인코딩 처리해서 집어넣는다. -> 1개에서 10000개를 케라스가 알아서 한다.
# 7자리 2자리 -> 임베딩이 자리를 덜 차지한다.
# 사전에서 사용된 단어 몇개? 답 : 5000개
# 그럼 원핫인코딩에 사용된 이진수는 몇개 ? 답 : 5000개. 5천개자리수. -> 메모리에 저장하기는 너무 비효율적이다. -> 소수 두개로해서 메모리 절약하자. -> compact하게 임베딩하자.
# 코드에서 임베딩 : 5천개에서 2개로 줄이기.
# 임베딩 층 안에도 학습하는 가중치가 있다. -> Google Word2Vec -> 임베딩웨이트메트릭스, 이 '행렬'이 학습된다.
# Google Word2Vec : pca와 내용이 굉장히 비슷하다.
# 예 : 5차원을 4차원으로 줄이고 싶어. (강의자료 예)
# 우리 코드에서는 5천 차원에서 32차원으로 -> 자료에서는 1만차원에서 32차원으로 줄이기.
# 어떻게 차원을 줄여주느냐 ? 답 : 행렬을 곱하면 된다.

# 임베딩 입력은 -> 원핫 벡터(1-hot-encoding된 라벨)가 들어온다 -> 원핫인코딩된 수가 들어온다.-> 네번쨰가 1이라 4번째 행이 출력된다.
# 행렬의 4행이 가중치고, 4행의 값이 변한다.
# 1만개에서 32개로 특징을 추출해서 함축한다 -> 임베딩행렬이 학습해서. 
# 자연어 처리에서 임베딩은 반드시 필요하다. 

학습용 입력 데이터 :  (25000, 80)
학습용 출력 데이터 :  (25000,)
평가용 입력 데이터 :  (25000, 80)
평가용 출력 데이터 :  (25000,)


In [ ]:
# 8번 셀

# RNN 구현
model = Sequential()

# 임베딩 층 추가
# 옵션이 세개 붙는다.
# input_dims : 
# input _length : 몇단이 있느냐
model.add(Embedding(input_dim=MY_WORDS,
                    output_dim=MY_EMBED,
                    input_length=MY_LEN))

# LSTM 추가 : 내가 다음 단으로 몇개를 넘겨줄지만 지정해주면 된다.
# LSTM이 몇단으로 지정되는지 어떻게 알아요? 답 : 알아서 해준다.
model.add(LSTM(units=MY_HIDDEN))

# 출력층 : 마지막으로 나가는 수 하나니깐 units=1
# 숫자하나의 확률 : sigmoid
model.add(Dense(units=1,
                activation='sigmoid'))

# RNN 요약 : 총 16만개 파라미터 갖는 RNN이 만들어졌다. -> Total params: 184,897가 정상
# 16만은 5천 곱 32 : 임베딩 행렬의 크기
# (None, 80, 32)  : 80단 32 출력?
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 80, 32)            160000    
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 184,897
Trainable params: 184,897
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 9번 셀

# RNN 학습

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc']) # metrics=['acc'] : 정확도도 보여달라고 하는 것.

print('학습시작')
begin = time()
model.fit(X_train,
          Y_train,
          epochs=MY_EPOCH,
          batch_size=MY_BATCH,
          verbose=1)


end = time()
print('총 학습 시간 : ', end - begin)

학습시작
Epoch 1/10
125/125 [==============================] - 8s 25ms/step - loss: 0.5239 - acc: 0.7205
Epoch 2/10
125/125 [==============================] - 3s 25ms/step - loss: 0.3318 - acc: 0.8590
Epoch 3/10
125/125 [==============================] - 3s 25ms/step - loss: 0.2904 - acc: 0.8791
Epoch 4/10
125/125 [==============================] - 3s 25ms/step - loss: 0.2642 - acc: 0.8919
Epoch 5/10
125/125 [==============================] - 3s 25ms/step - loss: 0.2469 - acc: 0.9001
Epoch 6/10
125/125 [==============================] - 3s 25ms/step - loss: 0.2259 - acc: 0.9099
Epoch 7/10
125/125 [==============================] - 3s 26ms/step - loss: 0.2014 - acc: 0.9217
Epoch 8/10
125/125 [==============================] - 3s 25ms/step - loss: 0.1776 - acc: 0.9331
Epoch 9/10
125/125 [==============================] - 3s 25ms/step - loss: 0.1564 - acc: 0.9404
Epoch 10/10
125/125 [==============================] - 3s 26ms/step - loss: 0.1358 - acc: 0.9523
총 학습 시간 :  36.7720160484314


In [ ]:
# 10번 셀

# RNN 평가하기 : 코드가 항상 똑같다
score = model.evaluate(X_test,
               Y_test)

print('최종 정확도 : ', score[1]) # 10번 학습해서 81점이면 나쁜건 아니다. 

# 80점대를 90점대로 끌어올릴려면? 
# 답안1: my_words = 100000 -> 결과:  no. 효과없음
# 답안2 : epoch 올리기 -> 학습데이터는 같다. 같은 데이터를 몇번 반복했느냐의 차이이다.epoch 올리면 이 차이다.
# 이코딩은 당장 써먹을 수 있다. -> 한글처리를 하면 한글도 사용가능하다.
# 에러 역추적 시 전역적을 봐야함.

782/782 [==============================] - 7s 8ms/step - loss: 0.5757 - acc: 0.8161
최종 정확도 :  0.8161200284957886


In [ ]:
# 11번 셀

# RNN으로 예측

pred = model.predict(X_test)

# 평가용 데이터 샘플 예측
# print('평가용 영화평 0번 : ', X_test[0] ) # 왜 0으로 꽉차있어? 답 : 앞을 0으로 패딩함. 자릿수맞추기 위해.
print('평가용 영화평 0번 : ', decode(X_test[0])) # 부정적 평가임을 확인한다. 그리고어덯게 예측했느지 본다.
print('정답 : ', Y_test[0]) 
print('예측 : ', pred[0]) # 예측이 고퀄 예측인가: 답 : 네. 0에 가까울 수록 0(부정)이고, 1에 가까울 수록 1(긍정)이므로 고퀄리티 예측이다. 거의 맞춘거다. 이정도면 성공이다.

['???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', 'please', 'give', 'this', 'one', 'a', 'miss', 'br', 'br', '???', '???', 'and', 'the', 'rest', 'of', 'the', 'cast', '???', 'terrible', 'performances', 'the', 'show', 'is', 'flat', 'flat', 'flat', 'br', 'br', 'i', "don't", 'know', 'how', 'michael', '???', 'could', 'have', 'allowed', 'this', 'one', 'on', 'his', '???', 'he', 'almost', 'seemed', 'to', 'know', 'this', "wasn't", 'going', 'to', 'work', 'out', 'and', 'his', 'performance', 'was', 'quite', '???', 'so', 'all', 'you', '???', 'fans', 'give', 'this', 'a', 'miss']
평가용 영화평 0번 :  None
정답 :  0
예측 :  [0.5331229]


In [ ]:
# 12번 셀

# 혼동행렬 : 혼동행렬 만들기 위해서는 예측값이 확률값이면 안된다. 부정 긍정으로 만들어줘야한다. 이진수로 만들어줘야한다. 
# 확률 결과
print(pred)

# 확률 결과를 를 이진수로 전환
# 확률값을 이진수로 바꾸는 코드는 이하와 같다.
pred = (pred > 0.5)
print(pred)

# 혼동행렬 찍기
print()
print()
print(confusion_matrix(Y_test,
                       pred))

# [[10243  2257]
#   [ 2340 10160]]
# 에서 10243은 TP
# 에서 -> 채우고 맞는지 이메일로 질문하자
# 에서
# 에서 10160 : 부정적인 평이 답인데 예측을 부정으로 해서 예측이 맞은 경우 
# 여기서 TN은 무슨 뜻인가 : 부정적인 평이 답인데 예측을 부정으로 해서 예측이 맞은 경우 
# False(사실) Negative(예측) : neg로 예측해서 틀린 경우 

# F1 점수
# average : 설명이 긴 관계로 생략한다. 
print(f1_score(Y_test,
               pred,
               average='micro')) # 성적표에 최종 정확도와 같다. 

[[ True]
 [ True]
 [False]
 ...
 [False]
 [False]
 [ True]]
[[ True]
 [ True]
 [False]
 ...
 [False]
 [False]
 [ True]]


[[10243  2257]
 [ 2340 10160]]
0.81612


In [ ]:
# 심장병 판별하기 우리 애저로 했다. -> 심장병 판별하기 문제 스킵한다.
# 나중에 시간 남으면 심장병 해주겠다. 


# 한줄한줄 전부 다 꼼꼼하게 가면 진도가 밀린다.
# 아직 설명 안한 것 : adam, LSTM